# Segmenting and Clustering Neighborhoods in Toronto

## <u> Task One: A data frame for Toronto neighborhoods </u> 

#### importing the required libraries

In [14]:
# First updating the libraries as required (code commented for earlier installs)

#!pip install -U numpy

#!pip install -U pandas

#!pip install -U scipy==1.4.1

#!pip install -U scikit-learn

#!pip install -U imbalanced-learn
print("libs updated")

libs updated


In [2]:
#import pandas library
import pandas as pd # library for data analysis and manipulation. will convert data to DataFrame objects 

# import requests
import requests # library to handle web requests

# import BeautifulSoup
from bs4 import BeautifulSoup  # library for pulling data out of HTML and XML files

####  defining the wikipedia data source and creating a Beautifulsoup object

In [15]:
# defining and retreiving the URL and creating a Beautifulsoup object 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

tn_data = requests.get(url).text

soup = BeautifulSoup(tn_data)

# print(soup.prettify()) # displaying the Beautifulsoup object contents


#### extracting table data from the soup object (the nested data structure document above)

In [4]:
# Creating an empty list to hold table data 
table_data=[]

# finding the table - and table data
table=soup.find('table')

# creating a dictionary called t_Cols having 3 keys PostalCode, Borough and Neighborhood

# ...and ignoring cells with a borough that is **Not assigned**
for row in table.findAll('td'):
    t_Cols = {}
    if row.span.text=='Not assigned':           # ignoring cells with a borough that is **Not assigned**
        pass
    else:
        t_Cols['PostalCode'] = row.p.text[:3]
        t_Cols['Borough'] = (row.span.text).split('(')[0]
        t_Cols['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_data.append(t_Cols)  # appending the dictionary data to the list object

# print(table_data)    # viewing the table data obtained


# converting the list object into a pandas data frame object
df=pd.DataFrame(table_data)

# cleaning Borough Names
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
# The data frame
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### Number of rows

In [6]:
no=df.shape[0]
print("The resultant dataframe has {} rows".format(no))


The resultant dataframe has 103 rows


## <u> Task Two: getting the latitude and the longitude coordinates of each neighborhood </u>

##### installing amd importing the required modules and libraries

In [7]:
# The Canada postal codes which have alphabets are not easily parsed in the Geocoder library 
# and we use the pgeocode module to fetch the latitude and longitude values

!pip install pgeocode
import pgeocode

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


##### fetching the geographical coordinates of a given postal code with pgeocode using a looping function untill cordinates for each postal code are found

In [8]:
#pgeocode.Nominatim('ca')

# defining a pgeocode instance for canada addresses
geolocator = pgeocode.Nominatim('ca')

# defining the list of postal addresses from our main dataframe
postal_codes = df['PostalCode'].tolist()

# initializing empty lists to store the latitudes and longitudes values
latitudes = []
longitudes = []

# looping through each postal code to get the addresses
for i, postal_code in enumerate(postal_codes):
    
    # Running a geo-localisation query for each Postal Code 
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        # storing the latitude and longitudes values from the query to the lists created earlier 
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)    

##### converting the pair of latitudes and longitudes lists into a pandas dataframe for confirmation 

In [9]:
lists = [latitudes, longitudes]
df_coords = pd.concat([pd.Series(x) for x in lists], axis=1)
df_coords.columns=['latitudes', 'longitudes'] 
df_coords.head(2)

,latitudes,longitudes
0,43.7545,-79.3300
1,43.7276,-79.3148


##### attaching the two as new columns into our main dataframe of  Toronto neighborhoods

##### This is the final dataframe for task two

In [10]:
df['Latitude']=df_coords['latitudes']
df['Longitude']=df_coords['longitudes']
pd.options.display.max_rows = None
print(df.shape)
df

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
7,M3B,North York,Don Mills North,43.7450,-79.3590
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


# <u> Task 3: Exploring and clustering the neighborhoods in Toronto </u>

##### 3.1 Here we will be working with only the boroughs whose names containing the word 'Toronto'

In [11]:
# filtering out the boroughs with the name "Toronto" into a new dataframe - df_viz
df_viz =df[df['Borough'].str.contains('Toronto')]

# reseting the index for the new dataframe
df_viz.reset_index(inplace=True, drop=True)

# size of df_viz 
print(df_viz.shape)
df_viz

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
2,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
3,M4E,East Toronto,The Beaches,43.6784,-79.2941
4,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754
5,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860
6,M6G,Downtown Toronto,Christie,43.6683,-79.4205
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6496,-79.3833
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6655,-79.4378
9,M4J,East York/East Toronto,The Danforth East,43.6872,-79.3368


we see that the new dataframe has 39 neighborhoods

##### 3.2 Next, it's segmenting and Clustering Neighborhoods in selected Boroughs

 ###### We start by importing the required libraries

In [12]:
import numpy as np # library to handle data in a vectorized manner
print("imported numpy")

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("imported pandas")

import json # library to handle JSON files
print("imported json")

!conda install -c conda-forge geopy --yes # code commented if installed earlier
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print("installed geopy")

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
print("imported requests")

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print("imported matplotlib")

# import k-means for clustering stage
from sklearn.cluster import KMeans
print("imported Kmeans")

!conda install -c conda-forge folium=0.5.0 --yes # code commented if installed earlier
print("installed folium")

import folium # map rendering library
print("imported folium")

print('Libraries updated/installed. All imported!')

imported numpy
imported pandas
imported json
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3

##### 3.3 Creating a map of Toronto with neighborhoods superimposed on top.
##### 3.3.1 First we get the latitude and longitude for  Toronto

In [13]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto City are 43.6534817, -79.3839347.


##### 3.3.2 Then we create a geo-map visualization of all the neighborhoods selected above (i.e, those with boroughs containing the name 'Toronto')  

In [14]:
# create map of Toronto using latitude and longitude values
neighborhoods = df_viz
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map for each neighborhood
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  

# display the map    
map_Toronto

Above is the map of Toronto for all neighborhoods with borough names containing the word Toronto

#### 3.4 Further more exploration of the selected neighborhoods
#### 3.4.1 Defining the  Foursquare Credentials and Version to be used in api calls  

In [15]:
# @hidden
CLIENT_ID = 'BML5E42WUO15B0NWXPECBYDTM24OEM1Z5TLTGSXWHL302ORB' #  Foursquare ID
CLIENT_SECRET = 'C2KDHSGGXQ02XG12NWAC1KKMTYPJ55QV2HJ2LOZNEXDYQ52H' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#print('My credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

##### 3.4.2 Let's explore the first neighborhood in our dataframe.

###### 3.4.2.1  <i> .The neighborhood's name....</i>


In [45]:
#getting the nieghborhood name ar the first row of our data

neighborhoods.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

###### 3.4.2.2. <i> The neighborhood's latitude and longitude values.

In [17]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6555, -79.3626.


###### 3.4.2.3  <i>  Getting the top 100 venues that are in Regent Park & Harbourfront within a radius of 500 meters.

<i>  First, creating the GET request URL. Name of URL is url.


In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# creating URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # displaying the URL

'https://api.foursquare.com/v2/venues/explore?&client_id=BML5E42WUO15B0NWXPECBYDTM24OEM1Z5TLTGSXWHL302ORB&client_secret=C2KDHSGGXQ02XG12NWAC1KKMTYPJ55QV2HJ2LOZNEXDYQ52H&v=20180605&ll=43.6555,-79.3626&radius=500&limit=100'

<i> Second, sending the GET request and examining the resutls

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60af8064e66f2f772397b8ec'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 21,
  'suggestedBounds': {'ne': {'lat': 43.660000004500006,
    'lng': -79.3563918719477},
   'sw': {'lat': 43.6509999955, 'lng': -79.36880812805231}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'di

<b> We see that all the information is in the items key. </b>
- The information for the first venue  is replicated here:

In [20]:
venue_1 = results['response']['groups'][0]['items'][0]
venue_1

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '53b8466a498e83df908c3f21',
  'name': 'Tandem Coffee',
  'location': {'address': '368 King St E',
   'crossStreet': 'at Trinity St',
   'lat': 43.65355870959944,
   'lng': -79.36180945913513,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65355870959944,
     'lng': -79.36180945913513}],
   'distance': 225,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['368 King St E (at Trinity St)',
    'Toronto ON',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-53b8466a498e83

<i> we define the get_category_type function from the Foursquare lab here.

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

 <i> Cleaning the json and structure into a pandas dataframe.

In [22]:
# defining the venues data path 
venues = results['response']['groups'][0]['items']
 
# converting the venues data to pandas 
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns of interest
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row using the function defined in cell above
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# cleaning column names
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# displating the venues near 
nearby_venues.head()

,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,The Yoga Lounge,Yoga Studio,43.655515,-79.364955
4,Body Blitz Spa East,Spa,43.654735,-79.359874


<i> Number of venues  returned by Foursquare

In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

21 venues were returned by Foursquare.


##### 3.5. Similar Exploration of all other Neighborhoods in out Toronto dataframe
###### 3.5.1 <i>  a function to repeat the same process  to all the neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

###### <i> 3.5.2 Now  the code to run the above function on each neighborhood and create a new dataframe called Toronto_venues

In [25]:
Toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

###### <i> 3.5.3 checking the size of the resulting dataframe

In [26]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1514, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa


1,514 venues were returned for the 39 neighborhoods of inhterest

###### <i> 3.5.4 checking how many venues were returned for each neighborhood

In [27]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,94,94,94,94,94,94
"Brockton, Parkdale Village, Exhibition Place",39,39,39,39,39,39
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",57,57,57,57,57,57
Central Bay Street,59,59,59,59,59,59
Christie,12,12,12,12,12,12
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,23,23,23,23,23,23
Davisville North,8,8,8,8,8,8


###### <i> 3.5.5 finding out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} unique categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 218 unique categories.


##### 3.6 Analyzing Each Neighborhood

In [29]:
# one hot encoding :  Creating a dataframe named "Toronto_onehot" whose columns are the didifferent  venue categories
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to "Toronto_onehot" dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
first_column = Toronto_onehot.pop('Neighborhood')
Toronto_onehot.insert(0, 'Neighborhood', first_column)


Toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Above is, the number of occurences for each unique type of venue is displayed per neighborhood with neighborhood rows replicated depending on the different venues they host

###### 3.6.1 <i> examining the new dataframe size.

In [30]:
Toronto_onehot.shape

(1514, 218)

###### <i> 3.6.2  Next, grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.053191,0.000000,0.000000,0.000000,0.010638,0.000000,0.031915,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.042553,0.00000,0.000000,0.000000,0.021277,0.000000,0.000000,0.00000,0.021277,0.042553,0.127660,0.000000,0.000000,0.00,0.00,0.010638,0.000000,0.010638,0.00,0.000000,0.000000,0.000000,0.010638,0.000000,0.000000,0.00000,0.000000,0.021277,0.010638,0.000000,0.010638,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.010638,0.000000,0.010638,0.000000,0.000000,0.00,0.000000,0.010638,0.010638,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.010638,0.000

In [32]:
 # confirming the new size
Toronto_grouped.shape

(39, 218)

###### <i> 3.6.3  Let's print each neighborhood along with the top 5 most common venues


In [33]:
num_top_venues = 5

for hood in Toronto_grouped ['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.13
1               Hotel  0.05
2              Bakery  0.05
3        Cocktail Bar  0.04
4  Seafood Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.08
2          Breakfast Spot  0.05
3  Thrift / Vintage Store  0.05
4                     Bar  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2                Café  0.05
3                 Bar  0.05
4           Speakeasy  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1      Sandwich Place  0.05
2                Café  0.05
3          Restaurant  0.03
4  Italian Restaurant  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  

###### <i> 3.7 Let's put that into a _pandas_ dataframe

<b> 3.7.1 First, let's write a function to sort the venues in descending order.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<b> 3.7.2 Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Bakery,Cocktail Bar,Seafood Restaurant,Café,Japanese Restaurant,Beer Bar,Restaurant,Pharmacy
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Thrift / Vintage Store,Bar,Gift Shop,Mexican Restaurant,Boutique,Brewery,Liquor Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,Bar,Speakeasy,Bank,Park,French Restaurant,Gym / Fitness Center,Grocery Store
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Electronics Store,French Restaurant,Breakfast Spot
4,Christie,Grocery Store,Café,Playground,Coffee Shop,Baby Store,Park,Athletics & Sports,Candy Store,Moroccan Restaurant,Monument / Landmark


Above we displayed the ten types unique venues per neighborhood in order of popularity 

##### 3.8 Clustering Neighborhoods


###### <i> 3.8.1  Run k-means to cluster the neighborhoods into 5 clusters.

In [36]:
# set number of clusters
kclusters = 5

# dropping the "Neighborhood" column that is a string and not required in clustering
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 4, 0, 0, 0, 0, 0], dtype=int32)

###### <i> 3.8.2 Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_viz

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.tail() # check the last 5 columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,M5W,Downtown Toronto Stn A,Enclave of M5E,43.6437,-79.3787,0,Coffee Shop,Restaurant,Hotel,Gym,Deli / Bodega,Café,Japanese Restaurant,Sporting Goods Shop,Italian Restaurant,Park
35,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.6684,-79.3689,0,Coffee Shop,Restaurant,Bakery,Pizza Place,Italian Restaurant,Pet Store,Café,General Entertainment,Market,Bus Stop
36,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.6492,-79.3823,0,Coffee Shop,Hotel,Café,Restaurant,Gym,Deli / Bodega,Salad Place,Seafood Restaurant,Steakhouse,Japanese Restaurant
37,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830,0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Gym,Fast Food Restaurant,Burger Joint,Men's Store
38,M7Y,East Toronto Business,Enclave of M4L,43.7804,-79.2505,0,Coffee Shop,Restaurant,Japanese Restaurant,Salad Place,Bookstore,Sushi Restaurant,Bank,Breakfast Spot,Yoga Studio,Intersection


Above we show which cluster each of the neighborhoods fall into

###### <i> 3.9 Finally, let's visualize the resulting clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# Display the clusters
map_clusters

The resulting clustering visual shows that most of the neighborhoods are very similar with the rest of the clusters comprising very few neighbourhoods each. 

<b><i><font color=red> if map is not showing you can try this nbviewer link to the same Github file </font> </i></b>  
https://nbviewer.jupyter.org/github/Karumaj/Coursera_IBMADS_Capstone/blob/master/ADSC_Toronto_Neighborhoods.ipynb


## 3.10. Examining Clusters
Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster..


###### <i> 3.10.1 Cluster 1

In [39]:
cluster_1 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
print(cluster_1.shape)
cluster_1

(32, 12)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Spa,Event Space,Beer Store,Restaurant,Food Truck,Thai Restaurant
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Hotel,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Lingerie Store,Theater
2,Downtown Toronto,0,Coffee Shop,Café,Seafood Restaurant,Cocktail Bar,Gastropub,Italian Restaurant,Moroccan Restaurant,Cheese Shop,Theater,Park
3,East Toronto,0,Pub,Health Food Store,Trail,Cheese Shop,Gastropub,Bakery,Coffee Shop,Park,Music Store,Middle Eastern Restaurant
4,Downtown Toronto,0,Coffee Shop,Hotel,Bakery,Cocktail Bar,Seafood Restaurant,Café,Japanese Restaurant,Beer Bar,Restaurant,Pharmacy
5,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Electronics Store,French Restaurant,Breakfast Spot
7,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Hotel,Gym,Japanese Restaurant,Steakhouse,Salad Place,Thai Restaurant,Asian Restaurant
8,West Toronto,0,Grocery Store,Bakery,Park,Bus Line,Furniture / Home Store,Liquor Store,Café,Bar,Bank,Gym
9,East York/East Toronto,0,Park,Coffee Shop,Intersection,Convenience Store,Rental Car Location,Greek Restaurant,Music Store,Museum,Movie Theater,Moroccan Restaurant
10,Downtown Toronto,0,Music Venue,Harbor / Marina,Café,Park,Accessories Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant


###### <i> 3.10.2 Cluster 2

In [40]:
cluster_2 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
print(cluster_2.shape)
cluster_2

(1, 12)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1,Home Service,Accessories Store,New American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant


###### <i> 3.10.3 Cluster 3

In [41]:
cluster_3 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
print(cluster_3.shape)
cluster_3

(1, 12)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Photography Studio,Park,Accessories Store,Music Store,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant


In [42]:
cluster_4 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
print(cluster_4.shape)
cluster_4

(1, 12)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,3,Park,Residential Building (Apartment / Condo),Accessories Store,Music Store,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant


###### <i> 3.10.5 Cluster 5

In [43]:
cluster_5 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
print(cluster_5.shape)
cluster_5

(4, 12)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,4,Grocery Store,Café,Playground,Coffee Shop,Baby Store,Park,Athletics & Sports,Candy Store,Moroccan Restaurant,Monument / Landmark
23,Central Toronto,4,Playground,Gym Pool,Park,Garden,Music Store,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
29,Central Toronto,4,Park,Playground,Thai Restaurant,Gym,Grocery Store,Trail,Molecular Gastronomy Restaurant,Movie Theater,Moroccan Restaurant,Monument / Landmark
33,Downtown Toronto,4,Playground,Grocery Store,Park,Candy Store,Music Venue,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant


#### 3.11 CLUSTERING RESULTS
Some general observations can be made about the uniqueness of the five resulting clusters as follows
##### cluster_1
<i> Above is the cluster with most neighborhoods - 32 out of 39 neighborhoods. From the abobe map, the neighbourborhoods occupy the coastline and are more concentrated on the central region of the city and are from all the boroughs in our exploration.

With cafes and eating places appering in most popular positions, they definitely represent the most urban places where most of the working population are found.</i>

##### cluster_2,3, & 4
<i> The three clusters are is only defined by one neighborhood each.
   They are in the outlying region of the city.
   They are probably surburban areas with charasteristic venues such as home service, accessories, studios and parks being most popular. </i>
   
##### cluster_5
Cluster 5 exhibits very common characterics among the neigborhoods that could be categorized as peri-urban neighborhoods  in relation to the other four since geographically from the map visual they appear spread in the areas between the core city neighborhoods and the outlying regions. They are probably mainly residential areas due to the common popularity of playgrounds, gym and grocery venues. 